# Text Mining Group Project


Alternative idea: maybe make model input a list of ingredients (based on what you have at home), and then let the model generate a reicpe for you based on the ingredients you have available

if that is the route we take, we can alternatively make another function that generates random lists of ingredients, and then feed the random ingredients to the recipe maker based on ingredients

### Imports

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Data Acquisition

In [6]:
import json
#https://eightportions.com/datasets/Recipes/#fn:1 where the data is from
#scraped from 
#Foodnetwork.com, Epicurious.com, Allrecipes.com by Ryan Lee

allr = open('recipes_raw_nosource_ar.json')
epi = open('recipes_raw_nosource_epi.json')
food = open('recipes_raw_nosource_fn.json')

data1 = json.load(allr) #data 1 has a lot of the word ADVERTISEMENT
data2 = json.load(epi) #data 2 looks good
data3 = json.load(food) #this too

#this load module loads the data into a dictionary 


In [12]:
#Example of the double dict structue of the .json files:
print(data3["p3pKOD6jIHEcjf20CCXohP8uqkG5dGi"]['title'])

Grammie Hamblet's Deviled Crab


# Data processing

For this part, we really have to look at getting all of the recipes into the same format: so the model will learn to recognize the format, and start to use it.

Our current recipes are in a dict of list of dict form, where the first dict contains the recipe code as key and the recipe as value, and the recipe value itself is a list of dicts in the shape [{instructions: value}, {ingredients: value}, {instructions:value}, {title:value}]. This needs to be processed to be more workable


We start by defining a list of all the recipe codes

In [8]:
codes = list(data1.keys())

We can then use this to display a specific recipe by iterating over its respective code:

In [11]:
data1[codes[5]]

{'title': 'Banana Banana Bread',
 'ingredients': ['2 cups all-purpose flour ADVERTISEMENT',
  '1 teaspoon baking soda ADVERTISEMENT',
  '1/4 teaspoon salt ADVERTISEMENT',
  '1/2 cup butter ADVERTISEMENT',
  '3/4 cup brown sugar ADVERTISEMENT',
  '2 eggs, beaten ADVERTISEMENT',
  '2 1/3 cups mashed overripe bananas ADVERTISEMENT',
  'ADVERTISEMENT'],
 'instructions': 'Preheat oven to 350 degrees F (175 degrees C). Lightly grease a 9x5 inch loaf pan.\nIn a large bowl, combine flour, baking soda and salt. In a separate bowl, cream together butter and brown sugar. Stir in eggs and mashed bananas until well blended. Stir banana mixture into flour mixture; stir just to moisten. Pour batter into prepared loaf pan.\nBake in preheated oven for 60 to 65 minutes, until a toothpick inserted into center of the loaf comes out clean. Let bread cool in pan for 10 minutes, then turn out onto a wire rack.\n',
 'picture_link': 'jRnWGDXDdyOg3rta4/HVAR2rD19XubC'}

# Language Recognition

With some help from: [TowardsDataScience](https://towardsdatascience.com/text-generation-with-python-and-gpt-2-1fecbff1635b)

### Sequence
Here we define an arbitrary recipe string sequence to initially train and test our model with. This text will be improved when we have acquired some data, but this will do for initial development:

In [5]:
sequence = """Ingredients: 3 tomatoes, garlic, spaghetti, Spanish chili's, cheese. Boil the pasta, then rinse. Cut chili's and mince garlic, add to pan
Add tomatoes and the pasta, and grate a generous amount of cheese
Serve with a touch of basil and a glass of good red wine
           """




#Allemaal van Sergio

"""
seqdata = []
print(len(data3.keys()))
count = 0
for key, value in data3.items():
    check = False
    
    #check whether it has the key ingredients
    if "ingredients" in value.keys():
        check = True
    
    #amount of recipes you want
    if count <= 800 and check:
        count = count +1
        listingr = "ingredients: "
        for i in value["ingredients"]:
            listingr = listingr + i + " "
        if type(value["instructions"]) == str:
            r = listingr + value["instructions"]
            seqdata.append(r)
            continue

print(type(seqdata))
#first two recipes
print(seqdata[2])

"""

"placeholder"

'placeholder'

In [6]:
#We use the pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

GPT2's development can be found at: Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2018). Language Models are Unsupervised Multitask Learners. 24.

In [7]:
#Tokenizing the inputs
#print(seqdata[1])
inputs = tokenizer.encode(sequence, return_tensors='pt')
#print(inputs)



#Allemaal van Sergio

"""
tokenizer.pad_token = "tokenizer.eos_token"
#padding and truncation to make all tensors same size.
#encoded_choices = [tokenizer.encode(s) for s in seqdata]
input_ids = []

for i in seqdata:
    input_ids.append(tokenizer.encode(i, return_tensors='pt', padding=True,truncation = True))

print(input_ids) #input Ids is now list with tensors of each datapoint

"""
'placeholder'

'placeholder'

# Language Generation

Here we generate a recipe based on the encoded cooking texts that we fed the model above. We will use the model.generate() function to do this.
Model.generate() has a lot of different variable options worth looking at, which can be used to optimize our model. For instance, the temperature variable can be tweaked between 0-5 for increased randomness, we can try no_repeat_ngram_size=2 to prevent repitition, or tweak top_k. We'll just have to play around a bit and see what works best

In [ ]:
import pandas as pd
import numpy as np
# We set the output length to 500 tokens
EncodedRecipe = model.generate(inputs, max_length=500, do_sample=True)



#Allemaal van Sergio
"""
#this only works only when you pick a specific tensor of input ids and not for all
EncodedRecipe2 = model.generate(input_ids[2], max_length=1024, do_sample=True)
print(EncodedRecipe2)
"""

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
#Decode the tokenized outputs
#recipe = tokenizer.decode(EncodedRecipe[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
#print(recipe[259:])

#Decode the tokenized outputs
recipe = tokenizer.decode(EncodedRecipe[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
print(len(recipe))
print(recipe) #part of it original and part of it the same as before

this repetition in this recipe is a great example of why we may have to try ways to prevent repetition in the model generation

## Saving Recipes

In [ ]:
#Defining a function to save generated recipes as .txt files:
def save(recipe, filename):
    text_file = open(filename, "w", encoding = 'utf8')
    n = text_file.write(recipe)
    text_file.close()

In [ ]:
#Saving the previously generated recipe:
save(recipe[259:], 'FirstRecipe.txt')
#Note: I am leaving out the 259 context characters

# Reinforcement learning
I had some inspiration for a potentially novel approach to take this model in: reinforcement learning. While the current model already performs relatively well on creating sensical recipe texts, the model still has absolutely no idea of 'taste', and which ingredients work well together and which don't. Therefore, it may be desireable to give the model some sort of loss function score on its recipes so it knows which recipes are good and which are bad, so the model can improve.
We could do this by either reading the recipes and rating them on how good we think they might be, or maybe even try to cook some of the things our model suggests and see how well they work, so we can help the model improve. While this may take some time and make the coding harder, it could be really cool to have a recipe generation model trained and optimized on actual real-world cooking, and I can hardly imagine anyone has ever done something like that before. Who knows, maybe the model will actually end up becoming a really good cook

Contributors: Emilia, Sergio, Tim